# Examples on using MTSDataModel

## Load modules

In [1]:
import sys
sys.path.insert(0,r'../src/')
import MTSDataModel as mts

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
import rpy2

## Get data

In [3]:
filename = 'sample_data.csv'
colnames = {'level1': 'Variable', 'level2': 'Country', 'value': 'Value','index': 'Date'}
do = mts.MTSDataModel(filepath = filename, colnames = colnames)

In [4]:
df = do.ReturnDf()
df.head()

GDP                Inflation                       StockPrices  \
           DEU           FIN        AUT        DEU        FIN         AUT   
1970-01-01 NaN  6.355725e+10  25.154467  31.620100  12.784633   11.314692   
1970-04-01 NaN  6.568328e+10  25.412633  31.900933  12.861467   11.390358   
1970-07-01 NaN  6.706184e+10  25.781500  31.947800  12.946200   12.117531   
1970-10-01 NaN  6.790094e+10  26.002767  32.252033  13.017667   12.248969   
1971-01-01 NaN  6.437505e+10  26.260967  33.024367  13.290433   12.077389   

                                 
                  DEU       FIN  
1970-01-01  10.873145  1.247428  
1970-04-01   9.807037  1.306829  
1970-07-01   9.271091  1.348410  
1970-10-01   8.835394  1.401871  
1971-01-01   9.699077  1.490974

## Pre-processing

### Deflate

In [5]:
do.DeflateVariables(['StockPrices'], infvar = 'Inflation')

C:\Users\ville_000\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
../src\MTSDataModel.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame['X_hat'] = self.Deflate(frame)
C:\Users\ville_000\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).renam

### De-trend

In [6]:
do.DetrendVariables(['StockPrices_def','GDP'], difftype = 'ld')

## Feature engineering

### Wavelet MRA decompositions

In [ ]:
do.MRADecomposition(variables = ['GDP_ld1'], entities=['FIN'],levels = 6,expanding='none')
#do.MRADecomposition(variables = ['GDP_ld1'],levels = 6,expanding='none') # bug here

### Dimension reduction

In [ ]:
do.ReduceVariableDimension(suffix = 'PC', variables = ['StockPrices_def_ld1','GDP_ld1'] , entities = ['FIN','DEU'])

### Sum variables

In [ ]:
#variables = {"gdp+stock": ['GDP_ld1','StockPrices_def_ld1']}
#do.SumVariables(variables)

#df = do.ReturnDf()
#df.head()

In [37]:
class MyException(Exception):
    pass

def EntitiesDefault(variables):
    """
    Helper function to extract entities for which 
    all chosen variables exist.
    """
    entities = []
    # All possible entities
    all_entities = list(np.unique(df.columns.get_level_values(1).values))
    # Loop over all_entities
    for crtentity in all_entities:
        # If variables belong to list of all variables under current entity,
        # append current entity to entities
        if set(variables).issubset(list(df.iloc[:, df.columns.get_level_values(1).isin([crtentity])].columns.get_level_values(0))):
            entities.append(crtentity)
    return entities

def VariablesCheck(variables,entities):
    """Helper function to check chosen variables exist for chosen entities."""
    for entity in entities:
        frame = df.iloc[:, (df.columns.get_level_values(0).isin(variables)) & (df.columns.get_level_values(1) == entity)].copy()       
        if set(variables).issubset(frame.columns.get_level_values(0)) == False:
            raise MyException("Not all variables present for " + entity + ".")             

def SumVariables(df,variables,name,entities=None):
    """
    Aggregate given variables, under same entity, using simple sum.

    variables is a dict with key being new variable name and value designating
    variables to be summed together.
    """
    # If no entities selected, get those for which all given variables exists
    if entities == None:
        entities = EntitiesDefault(variables)
    # If entities selected, check that all variables exist for them
    else:
        VariablesCheck(variables,entities)

    for entity in entities:
        crt_frame = df.iloc[:, (df.columns.get_level_values(0).isin(variables)) & (df.columns.get_level_values(1) == entity)]
        crt_frame[name, entity] = crt_frame.sum(axis=1,skipna = False)
        df = pd.merge(df, crt_frame[name, entity], left_index = True, right_index = True, how = 'left')
    return df

GDP                Inflation                     StockPrices  \
           DEU           FIN        AUT      DEU        FIN         AUT   
1970-01-01 NaN  6.355725e+10  25.154467  31.6201  12.784633   11.314692   

                                StockPrices_def                       \
                  DEU       FIN             AUT        DEU       FIN   
1970-01-01  10.873145  1.247428       11.314692  10.873145  1.247428   

           StockPrices_def_ld1         GDP_ld1      
                           AUT DEU FIN     DEU FIN  
1970-01-01                 NaN NaN NaN     NaN NaN

In [46]:
df = do.ReturnDf()
df.head(1)

variables = ['GDP_ld1','StockPrices_def_ld1']
name = "gdp+stock"
#entities = None
entities = ['FIN']

df = SumVariables(df.copy(),variables,name=name,entities=entities)
df.tail(2)

C:\Users\ville_000\Anaconda3\lib\site-packages\ipykernel\__main__.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


GDP               Inflation                         \
                     DEU           FIN       AUT         DEU        FIN   
2018-10-01  2.993535e+12  2.041390e+11  117.4735  111.655467  111.83850   
2019-01-01  3.009343e+12  2.048832e+11  116.7358  110.764450  111.57555   

           StockPrices                        StockPrices_def             \
                   AUT        DEU         FIN             AUT        DEU   
2018-10-01  130.353240  98.342199  108.417365       27.912391  27.849869   
2019-01-01  127.735845  97.972692         NaN       27.524779  27.968417   

                      StockPrices_def_ld1                       GDP_ld1  \
                  FIN                 AUT       DEU       FIN       DEU   
2018-10-01  12.393552           -0.081448 -0.101685 -0.098282  0.004956   
2019-01-01        NaN           -0.013984  0.004248       NaN  0.005267   

                     gdp+stock  
                 FIN       FIN  
2018-10-01  0.005274 -0.093009  
2019-01-01  0.003639       NaN

## Plot

In [ ]:
fig = plt.figure(figsize=(15,7))
ax = fig.add_subplot(1,1,1)
do.PlotVariables(variables=['StockPrices_def_ld1','GDP_ld1','PC','GDP_ld1_wl5'], entities=['FIN'], ax=ax)